<a href="https://colab.research.google.com/github/soumabkargit/Extract_Text_Image/blob/master/TesseractMethod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tesseract installation**

[https://github.com/tesseract-ocr/tesseract/wiki](https://)


In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!sudo apt-get install tesseract-ocr-eng
!pip install remotezip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,346 kB/s)
debconf: unable to initi

##Importing all useful dependencies

In [133]:
from matplotlib.image import imread
import numpy as np
from matplotlib import pyplot as plt
import os
import pytesseract
from pytesseract import Output
import matplotlib.patches as patches
from PIL import Image
import re
from pathlib import Path
from shutil import copyfile
from remotezip import RemoteZip
import json
import csv
# Technically not necessary in newest versions of jupyter
%matplotlib inline

In [134]:
url = 'https://expressexpense.com/large-receipt-image-dataset-SRD.zip'
lang = 'eng'
image_extract_path = "image_extract"
image_NO_extract_path = "image_NO_extract"
image_NO_read_text_path = "image_NO_read_text"
data = "data"
list_regex = []
regex_1 = r'^TOTAL(.)*(\:)*(.)*(\d)*'
regex_2 = r'^BALANCE(.)*(\:)*(.)*(\d)*'
regex_3 = r'^AMOUNT(.)*(\:)*(.)*(\d)*'
regex_amount = r'(\d)*'
list_regex.append(regex_1)
list_regex.append(regex_2)
list_regex.append(regex_3)
list_good_files = []
list_bad_files = []
list_word_total = []
candidat_good_total = []
candidat_bad_total = []
list_candidat_word_total = []
list_word_total_file = []
list_invoices = {}
receipt_ocr = {}
total_invoice_good_info = {}
total_invoice_bad_info = {}
all_total_invoice_info = {}

In [153]:
def hasNumbers(inputString):
  return any(char.isdigit() for char in inputString)

def add_regex(list_regex, regex):
  list_regex.append(regex)

def extract_amount(regex_amount, string):
  return re.search(regex_amount, string, re.IGNORECASE)

def createFolder(path):
  try:
    os.mkdir(path)
  except OSError:
    print ("Creation of the directory %s failed" % path)
  else:
    print ("Successfully created the directory %s " % path)

def extracted(url, data):
  with RemoteZip(url) as zip:
    zip.extractall(path=data)
  
def extracted_text_split(imageName, lang):
  im = np.array(Image.open(imageName), dtype=np.uint8)
  extracted_text = pytesseract.image_to_string(im, lang)
  text_split_line = extracted_text.splitlines()
  return text_split_line

def extracted_text_split_file_name(file_path):
  im = file_path
  text_split_text = extracted_text_split(im, lang)
  return text_split_text

def read_all_line_file(text_split_text):
  file_info = {}
  for line in text_split_text:
    for regex in list_regex:
      if regexLine(regex, line):
        if hasNumbers(line):
          file_info['good_info_candidat'] = line
        else:
          file_info['bad_info_candidat'] = line
  return file_info

def regexLine(regex, line):
  return re.search(regex, line, re.IGNORECASE)
    
def copy_file_folder(file_name, path):
  copyfile(file_name, path)

def get_all_names_files(data):
  try :
    list_files = os.listdir(data)
  except OSError:
    print ("Read all files in folder is %s failed" % data)
  else:
    print ("Successfully to read all files in the folder %s " % data)
  return list_files

def get_all_amount(data):
  
  files_infos_results = {}
  temp_list_files = get_all_names_files(data)

  if len(temp_list_files) == 0 :
    print("the folder is empty")
  else :
    for image_filename in temp_list_files:
      file_info_res = get_amount(image_filename)
      files_infos_results['Resume'] = file_info_res
      arrange_files_quality(file_info_res)
  return files_infos_results

def get_amount(file_name):

   file_info_result = {}

   if len(extracted_text_split_file_name(data + '/' + file_name)) == 0 :
     #print("the file has not been read")
     copy_file_folder(data + '/' + file_name, image_NO_read_text_path + '/' + file_name)
   else :
     text_split_text = extracted_text_split_file_name(data + '/' + file_name)

     if len(read_all_line_file(text_split_text)) == 0 :
       temp_file_info_result = "no information was read"
       #print("no information was read")
     else :
       temp_file_info_result = read_all_line_file(text_split_text)     

     file_info_result[file_name] = file_name
     file_info_result['Amount'] = temp_file_info_result
   arrange_files_quality(file_info_result)
   return file_info_result

def arrange_files_quality(file_info_res):
  try :
    file_info_res[file_name]
    copyfile(data + '/' + file_name, image_extract_path + '/' + file_name)
  except KeyError:
    print("the amount was not extracted correctly")
    try :
      file_info_res[file_name]
      copyfile(data + '/' + file_name, image_NO_extract_path + '/' + file_name) 
    except KeyError:  
      print("the file has not been read")
      copyfile(data + '/' + file_name, image_NO_read_text_path + '/' + file_name)
 
    


In [ ]:
createFolder(image_extract_path)
createFolder(image_NO_extract_path)
createFolder(image_NO_read_text_path)
createFolder(data)

In [6]:
extracted(url, data)

##Dataset

https://expressexpense.com/large-receipt-image-dataset-SRD.zip

## Reading of all invoice images and extraction of the total amount.

##Sort extracted and non-extracted images into respective folders.

In [154]:
file_name = '1199-receipt.jpg'
res = get_amount(file_name)

In [155]:
res

{'1199-receipt.jpg': '1199-receipt.jpg',
 'Amount': {'good_info_candidat': 'Total Due: 128.33'}}

In [158]:
res['Amount']['good_info_candidat']

'Total Due: 128.33'

In [ ]:
get_all_amount(data)

#References

https://nanonets.com/blog/receipt-ocr/?fbclid=IwAR0H4VkHhpimqKIgEUQOUCOJS1mtk6vQREcv3MMiqJ2NpxG9LHZnsHM-aeE


https://www.datacamp.com/community/tutorials/zip-file